## Importing Required Libraries

In [1]:
import pandas as pd
import numpy as np
from scipy import linalg, optimize
import re
import random
import math
import string
from nltk.corpus import wordnet as wn
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from nltk import word_tokenize
from scipy import optimize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from scipy.sparse import coo_matrix, hstack, vstack
import seaborn as sns
from textblob import TextBlob
# Graphing: 
%matplotlib inline


In [2]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive/BT4222 Data/training_set.csv"
path = '/content/drive/My Drive/BT4222 Data/training_set.csv'
training_set = pd.read_csv(path)

Mounted at /content/drive
'/content/drive/My Drive/BT4222 Data/training_set.csv'


In [3]:
!ls "/content/drive/My Drive/BT4222 Data/training_labels.csv"
path = '/content/drive/My Drive/BT4222 Data/training_labels.csv'
training_labels = pd.read_csv(path)

'/content/drive/My Drive/BT4222 Data/training_labels.csv'


In [4]:
!ls "/content/drive/My Drive/BT4222 Data/testing_set.csv"
path = '/content/drive/My Drive/BT4222 Data/testing_set.csv'
testing_set = pd.read_csv(path)

'/content/drive/My Drive/BT4222 Data/testing_set.csv'


In [5]:
!ls "/content/drive/My Drive/BT4222 Data/testing_labels.csv"
path = '/content/drive/My Drive/BT4222 Data/testing_labels.csv'
testing_labels = pd.read_csv(path)

'/content/drive/My Drive/BT4222 Data/testing_labels.csv'


In [6]:
!ls "/content/drive/My Drive/BT4222 Data/combined.csv"
path = '/content/drive/My Drive/BT4222 Data/combined.csv'
combined = pd.read_csv(path)

'/content/drive/My Drive/BT4222 Data/combined.csv'


## LightGBM

> With GridSearch

* Training Errors
> Mean Squared Error:  0.05236697071909393
> RMSE:  0.2288383069311035
> Mean Absolute Error:  0.05840637214862351
> R-squared:  0.9476325904944419
> Adjusted R2:  0.9476264467117445
* Test Errors
> Mean Squared Error:  0.05968673906035098
> RMSE:  0.24430869624381155
> Mean Absolute Error:  0.06415639168013557
> R-squared:  0.9335835121117316
> Adjusted R2:  0.9335133150084223

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error

# Initialise LGBMRegressor
# rf = LGBMRegressor(n_estimators=300, subsample=0.9, colsample_bytree=0.2, eval_metric = 'rmse')
rf = LGBMRegressor(n_estimators=300, eval_metric = 'rmse')


# Set up 5-fold cross-validation
from sklearn import model_selection
cv = model_selection.KFold(5)


# Pipeline Standardization and Model
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler())
                           , ('model', rf) ])
# Tuning the model
my_min_samples_leaf = [2, 10, 25, 50, 100]
my_max_depth = [7, 8, 9, 10, 11, 12]
my_subsample = [0.7, 0.8, 0.9]
my_colsample_bytree = [0.1, 0.2, 0.3]

# Run the model using GridSearch, select the model with best search
from sklearn.model_selection import GridSearchCV
optimized_rf = GridSearchCV(estimator=pipeline
                            , cv=cv
                            , param_grid =dict(model__min_samples_leaf = my_min_samples_leaf, 
                                               model__max_depth = my_max_depth,
                                               model__subsample = my_subsample,
                                               model__colsample_bytree = my_colsample_bytree)
                            , scoring = 'neg_mean_squared_error'
                            , verbose = 1
                            , n_jobs = -1
                           )

# Fitting on Training Data
optimized_rf.fit(training_set, training_labels)


# Best model estimators
print(optimized_rf.best_estimator_)

def adjusted_r2(r2, n, p):
  result = 1-((1-r2)*((n-1)/(n-p-1)))
  return result


# Evaluate metrics on holdout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_train_pred = optimized_rf.predict(training_set)
y_pred = optimized_rf.predict(testing_set)
n1 = len(y_train_pred)
p1 = training_set.shape[1]
n2 = len(y_pred)
p2 = testing_set.shape[1]
print('Training Errors')
print("Mean Squared Error: ", mean_squared_error(training_labels, y_train_pred))
print("RMSE: ", mean_squared_error(training_labels, y_train_pred, squared = False))
print("Mean Absolute Error: ", mean_absolute_error(training_labels, y_train_pred))
print("R-squared: ", r2_score(training_labels, y_train_pred))
print("Adjusted R2: ", adjusted_r2(r2_score(training_labels, y_train_pred), n1, p1))

print('Test Errors')
print("Mean Squared Error: ", mean_squared_error(testing_labels, y_pred))
print("RMSE: ", mean_squared_error(testing_labels, y_pred, squared = False))
print("Mean Absolute Error: ", mean_absolute_error(testing_labels, y_pred))
print("R-squared: ", r2_score(testing_labels, y_pred))
print("Adjusted R2: ", adjusted_r2(r2_score(testing_labels, y_pred), n2, p2))

# ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 19.5min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 31.3min
[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed: 33.9min finished
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 LGBMRegressor(boosting_type='gbdt', class_weight=None,
                               colsample_bytree=0.3, eval_metric='rmse',
                               importance_type='split', learning_rate=0.1,
                               max_depth=9, min_child_samples=20,
                               min_child_weight=0.001, min_samples_leaf=2,
                               min_split_gain=0.0, n_estimators=300, n_jobs=-1,
                               num_leaves=31, objective=None, random_state=None,
                               reg_alpha=0.0, reg_lambda=0.0, silent=True,
                               subsample=0.7, subsample_for_bin=200000,
                               subsample_freq=0))],
         verbose=False)
Training Errors
Mean Squared Error:  0.05236697071909393
RMSE:  0.2288383069311035
Mean Absolute Er

## XGBoost

> With Grid Search

* Training Errors
> Mean Squared Error:  0.006577338708038475
> RMSE:  0.08110079351053524
> Mean Absolute Error:  0.034764883555239344
> R-squared:  0.9934226061799862
> Adjusted R2:  0.9934218345153435
* Test Errors
> Mean Squared Error:  0.04780776840983414
> RMSE:  0.21864987630875563
> Mean Absolute Error:  0.04773988421430823
> R-squared:  0.9468018504353819
> Adjusted R2:  0.9467456240958149

In [8]:
from sklearn.ensemble import RandomForestRegressor
import xgboost
from sklearn import preprocessing
from xgboost import XGBRFRegressor

# Initialise XGBRegressor
rf = xgboost.XGBRegressor(n_estimators=300, subsample=0.9, colsample_bynode=0.2, n_fold = 5, eval_metric = 'rmse')

# Set up 5-fold cross-validation
from sklearn import model_selection
cv = model_selection.KFold(5)

# Pipeline standardization and Model
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler())
                           , ('model', rf) ])
# Tuning the model
my_min_samples_leaf = [2, 10, 25, 50, 100]
my_max_depth = [7, 8, 9, 10, 11, 12]


# Run the model using GridSearch, select the model with best search
from sklearn.model_selection import GridSearchCV
optimized_rf = GridSearchCV(estimator=pipeline
                            , cv=cv
                            , param_grid =dict(model__min_samples_leaf = my_min_samples_leaf, model__max_depth = my_max_depth)
                            , scoring = 'neg_mean_squared_error'
                            , verbose = 1
                            , n_jobs = -1
                           )

# Fitting on Training Data
optimized_rf.fit(training_set, training_labels)


# Best model estimators
print(optimized_rf.best_estimator_)

def adjusted_r2(r2, n, p):
  result = 1-((1-r2)*((n-1)/(n-p-1)))
  return result


# Evaluate metrics on holdout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_train_pred = optimized_rf.predict(training_set)
y_pred = optimized_rf.predict(testing_set)
n1 = len(y_train_pred)
p1 = training_set.shape[1]
n2 = len(y_pred)
p2 = testing_set.shape[1]
print('Training Errors')
print("Mean Squared Error: ", mean_squared_error(training_labels, y_train_pred))
print("RMSE: ", mean_squared_error(training_labels, y_train_pred, squared = False))
print("Mean Absolute Error: ", mean_absolute_error(training_labels, y_train_pred))
print("R-squared: ", r2_score(training_labels, y_train_pred))
print("Adjusted R2: ", adjusted_r2(r2_score(training_labels, y_train_pred), n1, p1))

print('Test Errors')
print("Mean Squared Error: ", mean_squared_error(testing_labels, y_pred))
print("RMSE: ", mean_squared_error(testing_labels, y_pred, squared = False))
print("Mean Absolute Error: ", mean_absolute_error(testing_labels, y_pred))
print("R-squared: ", r2_score(testing_labels, y_pred))
print("Adjusted R2: ", adjusted_r2(r2_score(testing_labels, y_pred), n2, p2))

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 40.6min finished


[16:03:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=0.2,
                              colsample_bytree=1, eval_metric='rmse', gamma=0,
                              importance_type='gain', learning_rate=0.1,
                              max_delta_step=0, max_depth=9, min_child_weight=1,
                              min_samples_leaf=2, missing=None,
                              n_estimators=300, n_fold=5, n_jobs=1,
                              nthread=None, objective='reg:linear',
                              random_state=0, reg_alpha=0, reg_lambda=1,
                              scale_pos_weight=1, seed=None, silent=None,


## Without normalisation of Price, but keeping normalised variables


> Testing RMSE is $373260. This is an extremely large jump compared to previous models on non-normalised datasets.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(combined.drop('price', axis=1),
                                                    combined['price'], 
                                                    test_size=0.1,
                                                    random_state=1)

In [ ]:
# Normalise remaining lease for training and test
X_train_lease_mean = X_train['remaining_lease'].mean()

X_train_lease_std = X_train['remaining_lease'].std()
X_train['remaining_lease'] = (X_train['remaining_lease'] - X_train_lease_mean)/X_train_lease_std

X_test['remaining_lease'] = (X_test['remaining_lease'] - X_train_lease_mean)/X_train_lease_std


# Normalise floor area for training and test
X_train_area_mean = X_train['floor_area_sqm'].mean()

X_train_area_std = X_train['floor_area_sqm'].std()
X_train['floor_area_sqm'] = (X_train['floor_area_sqm'] - X_train_area_mean)/X_train_area_std

X_test['floor_area_sqm'] = (X_test['floor_area_sqm'] - X_train_area_mean)/X_train_area_std

In [ ]:
y_train /= 100000
y_test /= 100000

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_log_error

# Initialise LGBMRegressor
# rf = LGBMRegressor(n_estimators=300, subsample=0.9, colsample_bytree=0.2, eval_metric = 'rmse')
rf = LGBMRegressor(n_estimators=300, eval_metric = 'rmse')


# Set up 5-fold cross-validation
from sklearn import model_selection
cv = model_selection.KFold(5)


# Pipeline Standardization and Model
from sklearn.pipeline import Pipeline
pipeline = Pipeline(steps=[('standardize', preprocessing.StandardScaler())
                           , ('model', rf) ])
# Tuning the model
my_min_samples_leaf = [2, 10, 25, 50, 100]
my_max_depth = [7, 8, 9, 10, 11, 12]
my_subsample = [0.7, 0.8, 0.9]
my_colsample_bytree = [0.1, 0.2, 0.3]

# Run the model using GridSearch, select the model with best search
from sklearn.model_selection import GridSearchCV
optimized_rf = GridSearchCV(estimator=pipeline
                            , cv=cv
                            , param_grid =dict(model__min_samples_leaf = my_min_samples_leaf, 
                                               model__max_depth = my_max_depth,
                                               model__subsample = my_subsample,
                                               model__colsample_bytree = my_colsample_bytree)
                            , scoring = 'neg_mean_squared_error'
                            , verbose = 1
                            , n_jobs = -1
                           )

# Fitting on Training Data
optimized_rf.fit(X_train, y_train)


# Best model estimators
print(optimized_rf.best_estimator_)


# Evaluate metrics on holdout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_train_pred = optimized_rf.predict(X_train)
y_pred = optimized_rf.predict(X_test)
print('Training Errors')
print("Mean Squared Error: ", mean_squared_error(y_train, y_train_pred))
#print("Mean Squared Log Error: ", mean_squared_log_error(training_labels, y_train_pred))
print("Mean Absolute Error: ", mean_absolute_error(y_train, y_train_pred))
print("R-squared: ", r2_score(y_train, y_train_pred))

print('Test Errors')
print("Mean Squared Error: ", mean_squared_error(y_test, y_pred))
#print("Mean Squared Log Error: ", mean_squared_log_error(testing_labels, y_train_pred))
print("Mean Absolute Error: ", mean_absolute_error(y_test, y_pred))
print("R-squared: ", r2_score(y_test, y_pred))

# ValueError: Mean Squared Logarithmic Error cannot be used when targets contain negative values.

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 10.0min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 18.3min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed: 29.0min
[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed: 31.5min finished


Pipeline(memory=None,
         steps=[('standardize',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('model',
                 LGBMRegressor(boosting_type='gbdt', class_weight=None,
                               colsample_bytree=0.3, eval_metric='rmse',
                               importance_type='split', learning_rate=0.1,
                               max_depth=7, min_child_samples=20,
                               min_child_weight=0.001, min_samples_leaf=2,
                               min_split_gain=0.0, n_estimators=300, n_jobs=-1,
                               num_leaves=31, objective=None, random_state=None,
                               reg_alpha=0.0, reg_lambda=0.0, silent=True,
                               subsample=0.7, subsample_for_bin=200000,
                               subsample_freq=0))],
         verbose=False)
Training Errors
Mean Squared Error:  16.57098283180115
Mean Absolute Error:  1.0027637343707159
R-s